In [1]:
from lib.func import fetch

### Prepare Data

In [2]:
# Split workflow into it's name and it's rules
def splitNameAndRules(row):
    name, rules = row.split('{')
    return [name, rules[:-1].split(',')]


# Build string rule into object rule
def buildRule(rule):
    idx = rule.find(':')
    final = (idx == -1)
    obj = {
        'category': None if final else rule[0],
        'op': '=' if final else rule[1],
        'num': None if final else int(rule[2:idx]),
        'next': rule if final else rule[idx + 1:]
    }
    return obj


# Build object workflows from string representation
def buildWorkflows(workflows):
    output = {}
    for row in workflows:
        name, rules = splitNameAndRules(row)
        output.setdefault(name, [])
        for rule in rules:
            output[name].append(buildRule(rule))

    return output


# Build individual part object from string
def buildPart(part):
    obj = {}
    items = part[1:-1].split(',')
    for item in items:
        key, val = item.split('=')
        obj.setdefault(key, int(val))
    return obj


# Build parts and put into array
def buildParts(parts):
    output = []
    for part in parts:
        obj = buildPart(part)
        output.append(obj)
    return output


# Convert string input into useful objects
def buildData(data):
    workflows, parts = [x.split('\n') for x in data.split('\n\n')]
    return [buildWorkflows(workflows), buildParts(parts)]

In [3]:
data = '''px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}'''

In [4]:
data = fetch(19)

### Functions

In [5]:
# Run workflow with part to get next workflow name
def getNextFlowName(flow, part):
    for i in range(len(flow) - 1):
        rule = flow[i]
        cat_val = part[rule['category']]

        if rule['op'] == '>':
            if cat_val > rule['num']:
                return rule['next']
        else:
            if cat_val < rule['num']:
                return rule['next']
    return flow[-1]['next']


# Sum all remaining part category ratings
def getPartRatingsSum(accepted):
    total = 0
    for part in accepted:
        total += part['x'] + part['m'] + part['a'] + part['s']
    return total


### Solution

In [6]:
def solve(data):
    workflows, parts = buildData(data)
    accepted = []
    for part in parts:
        name = 'in'
        while name not in 'AR':
            flow = workflows[name]
            name = getNextFlowName(flow, part)
        
        if name == 'A':
            accepted.append(part)
                
    return getPartRatingsSum(accepted)


In [7]:
print(f'Part 1: {solve(data)}')

Part 1: 332145
